In [1]:
import matplotlib.pyplot as plt
import matplotlib

%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2


import numpy as np
#%matplotlib inline

import os
from utils import *

import os

from utils_calcium import ProcessCalcium, get_reward_triggered_psth, plot_multi_session_psth_imshow, plot_psth

Autosaving every 180 seconds


/home/cat/.conda/envs/bmitools/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [8]:
####################################################
################## LOAD DATA #######################
####################################################


# ****************** MICE AVAILABLE ***********************
# M1 = [
#     'DON-011733',
#     'DON-014382',
#     'DON-014451',
#     'DON-014575',
#     'DON-014618',

# CA3 = [
#     'DON-012502',
#     'DON-014266',
#     'DON-014371',
#     'DON-015801',
#     'DON-015821',
# ]
# *********************************************************

#
#root_dir = '/media/cat/4TB/donato/andres/master_analysis/'
root_dir = '/media/cat/8TB/donato/bmi/'
animal_id = 'DON-011733'


#root_dir = '/media/cat/8TB/donato/bmi/'
#animal_id = 'DON-014618'
#animal_id = 'DON-014575'

#       
c = ProcessCalcium(root_dir,
                   animal_id)

#
c.remove_bad_cells = False

#
c.recompute_binarization = False  # IF TRue: must input correct params from bin notebooks
c.dff_min = 0.05                  # min %DFF for [ca] burst to considered a spike (default 5%) overwrites percentile threshold parameter
c.percentile_threshold = 0.99   # this is pretty fixed, we don't change it; we want [ca] bursts that are well outside the "physics-caused"noise
c.maximum_std_of_signal = 0.25     # if std of signal is greater than this, then we have a noisy signal and we don't want to binarize it

#
c.save_figures = False
c.load_sessions()

#
print ("....DONE...")


  0%|          | 0/8 [00:00<?, ?it/s]

remove bad cells:  True
...setting 2p parameters...


 12%|█▎        | 1/8 [00:02<00:18,  2.64s/it]


remove bad cells:  True
...setting 2p parameters...


 25%|██▌       | 2/8 [00:05<00:16,  2.70s/it]

...


remove bad cells:  True
...setting 2p parameters...


/home/cat/.conda/envs/bmitools/lib/python3.8/site-packages/binarize2pcalcium/binarize2pcalcium.py:582: RuntimeWarning: invalid value encountered in divide
  cumsum = cumsum/np.max(cumsum)
 88%|████████▊ | 7/8 [00:31<00:05,  5.10s/it]


remove bad cells:  True
...setting 2p parameters...


/home/cat/.conda/envs/bmitools/lib/python3.8/site-packages/binarize2pcalcium/binarize2pcalcium.py:582: RuntimeWarning: invalid value encountered in divide
  cumsum = cumsum/np.max(cumsum)
100%|██████████| 8/8 [00:37<00:00,  4.64s/it]


....DONE...


In [9]:
########################################################
########################################################
########################################################

# adds trial information; eventually should also add bursts etc...
c.fix_spreadsheet()

#


...done...


In [11]:
########################################################
########################################################
########################################################
# load the ensemble state, trial starts, and trial ends info

#c.session_id = 8
print (c.session_ids)
c.fix_spreadsheet_missing_vals = True

#
print (c.session_types)
c.load_spreadsheet_multi()

[20230302, 20230303, 20230304, 20230305, 20230306, 20230307, 20230308, 20230309]
['day0', 'bmi1', 'bmi2', 'bmi3', 'bmi4', 'bmi5', 'bmi6', 'bmi7']


100%|██████████| 8/8 [00:00<00:00, 386.72it/s]


In [12]:
# also save the res array in the same folder
c.load_best_matches()


self.best_matches:  [380  15   2   4]


In [19]:
from utils_calcium import compute_si_1D
import parmap

# 
n_tests = 500
window = 300
recompute = False

#
session_ids = np.arange(1,8,1)
for session_id in session_ids:

    # make directory to save spatial info
    dir_si = os.path.join(c.root_dir,
                                c.animal_id,
                                str(c.session_ids[session_id]),
                                'spatial_info')
    if os.path.exists(dir_si)==False:
        os.mkdir(dir_si)

    # iniatize the data based on the sesssion
    cells = c.sessions[session_id].F_upphase_bin
    cell_idxs = np.arange(cells.shape[0])

    # break up cell_idxs into 32 chunks to run in parallel
    cell_idxs_lists = np.array_split(cell_idxs, 32)

    #
    reward_times = c.reward_times[session_id]
    session_name = str(c.session_ids[session_id])

    # run the code
    parmap.map(compute_si_1D,
                cell_idxs_lists,
                cells,           # this is the calcium activivty in the session
                reward_times,
                root_dir,
                animal_id,
                session_name,
                window,
                n_tests,
                recompute,
                pm_processes=32,
                pm_pbar=True)



 38%|███▊      | 12/32 [00:04<00:07,  2.83it/s]


KeyboardInterrupt: 

In [40]:
########################################################
####### vISUALIZE SINGLE CELLS Oover threshold #########
########################################################

min_thresh = 2.
max_thresh = 10.5
min_si_thresh = 2.
max_si_thresh = 15

#
session_id = 7

#
ctr=0
for k in cell_idxs:
    fname = os.path.join(c.root_dir,
                        c.animal_id,
                        str(c.session_ids[session_id]),
                        'spatial_info',
                        str(k)+'.npz')
    
    #
    d = np.load(fname, allow_pickle=True)
    si = d['si']
    zscore = d['zscore']
    ave_map = d['ave_map']

    # filter out cells with low activity 
    if np.max(ave_map)<0.1:
        continue

    #
    if np.logical_and(zscore>min_thresh, zscore<max_thresh):
    #if np.logical_and(si>min_si_thresh, si<max_si_thresh):


        ax=plt.subplot(5,5,ctr+1)
        #print ("cell: ", k, " zscore: ", zscore, " avg_rate: ", avg_rate)
        
        t = np.arange(ave_map.shape[0])/30-10
        plt.plot(t,ave_map)
        plt.title(str(k))
        plt.show()

        # plot vertical red line at 300
        plt.plot([0,0],[0,np.max(ave_map)],'r--')

        # plot title with zscore
        plt.title(str(k)+', si: '+str(np.round(si,2))+ ", zscore: "+str(np.round(zscore,2)),
                  fontsize=10)

        # 
        if ctr<20:
            plt.xticks([])
        else:
            plt.xlabel("Time (sec)",fontsize=10)

        #
        ctr+=1

    if ctr==25:
        break

#
plt.suptitle(
            "Cells with zscore between "+str(min_thresh)+" and "+str(max_thresh)
            #"Cells with SI between "+str(min_si_thresh)+" and "+str(max_si_thresh)
             +"\n"+str(c.animal_id)+", session: "+str(c.session_ids[session_id]))
plt.show()

In [46]:
#################################################
### load the cells and plot distribution of si###
#################################################

# make color map in viridis to plot 8 lines
cmap = matplotlib.cm.get_cmap('viridis')
clrs = cmap(np.linspace(0, 0.9, 8))


#
plt.figure()
ax1=plt.subplot(121)
plt.title("Spatial Information")
plt.legend()
ax2=plt.subplot(122)
plt.title("Z-score")
plt.legend()

#
session_ids = np.arange(1,8,1)

#
si_array = []
zscore_array = []
ave_map_array = []
for ctr,session_id in enumerate(session_ids):
    #
    sis = []
    zscores = []
    ave_maps = []
    for k in cell_idxs:
        fname = os.path.join(c.root_dir,
                            c.animal_id,
                            str(c.session_ids[session_id]),
                            'spatial_info',
                            str(k)+'.npz')
        
        #
        si = np.load(fname)['si']
        zscore = np.load(fname)['zscore']

        #
        sis.append(si)
        zscores.append(zscore)

        #
        ave_map = np.load(fname)['ave_map']
        ave_maps.append(ave_map)



    #
    sis = np.array(sis)
    zscores = np.array(zscores)
    ave_maps = np.array(ave_maps)

    si_array.append(sis)
    zscore_array.append(zscores)
    ave_map_array.append(ave_maps)


    #
    y = np.histogram(sis, bins=np.arange(0,10,0.1))
    xx = y[1][:-1]
    yy = y[0]
    # smooth yy to make it look better
    yy = np.convolve(yy, np.ones(5)/5, mode='same')
    ax1.plot(xx,yy,c=clrs[ctr], label="bmi #: " + str(session_id))

    y = np.histogram(zscores, bins=np.arange(-5,6.5,0.1))
    xx = y[1][:-1]
    yy = y[0]
    # smooth yy to make it look better
    yy = np.convolve(yy, np.ones(5)/5, mode='same')
    ax2.plot(xx,yy,c=clrs[ctr], label=str(session_id))

#
ax1.legend()

#
plt.suptitle("Spatial Information and Z-score Distribution for animal: "+c.animal_id)

#
plt.show()


/tmp/ipykernel_200673/3317522745.py:6: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = matplotlib.cm.get_cmap('viridis')
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [47]:
# plot metrics as function of session
cell_id = 0

p = []
for k in range(len(si_array)):
    si = si_array[k][cell_id]
    p.append(si)

plt.figure()
plt.plot(p)
plt.title("Spatial Information for cell: "+str(cell_id))
plt.show()

In [42]:
##############################################################
############# EXAMPLE OF SINGLE CELL BURSTS ##################
##############################################################
if False:      
    cell_id = 0

    #
    d = np.load('/media/cat/8TB/donato/bmi/DON-014618/20230505/spatial_info/'+str(cell_id)+'.npz', allow_pickle=True)
    rate_map = d['rate_map']

    # make a time vector that repeats every 600 time points
    n_rewards = int(rate_map.shape[0]/(30.*20.))
    print ("n_rewards: ", n_rewards)
    t = np.arange(rate_map.shape[0])/30.

    #
    plt.figure()
    plt.plot(t,d['rate_map'])

    # relabel xticks to repeat every 600 time points
    labels = np.arange(0,n_rewards+1,1)
    labels_old = np.linspace(t[0],t[-1], len(labels))
    plt.xticks(labels_old,
            labels)

    # shade background every other labels index
    for k in range(len(labels)):
        if k%2==0:
            try:
                plt.axvspan(labels_old[k], labels_old[k+1], color='grey', alpha=.2)
            except:
                pass

    # plot a vertical dashed red line every 600 time points starting at 300
    for k in range(1,len(labels)):
        plt.plot([labels_old[k]-10, labels_old[k]-10], 
                [0,1], '--', c='red')

    #
    plt.ylim(0,1)
    plt.xlim(0,labels_old[-1])

    #
    plt.xlabel("Reward #")
    plt.ylabel("UPphase")
    plt.title("Cell: "+str(cell_id))

    #
    plt.show()


In [ ]:
##############################################################
############### BUILD CELL BURST CLASSIFIER ##################
##############################################################

In [125]:
##############################################################
######## COMPUTE REWARD TRIGGERED RESPONSE OF ROIs ###########
##############################################################
# 
from utils_calcium import plot_psth_rois_only

#
def init_fig():
    plt.figure(figsize=(20,12))
    axes = []
    for k in range(4):
        ax=plt.subplot(2,2,k+1)
        axes.append(ax) 

    return axes

# 
window = 300
start_cell=0
show_random = True
idx_cells = None
smoothing = True
plot_separately = False
c.plot_rois_only = True
c.cells_to_plot = c.best_matches
c.cells_to_plot = np.random.choice(np.arange(c.sessions[0].F_filtered.shape[0]),
                                   4,
                                   replace=False)
#
print ("c.cells_to_plot: ", c.cells_to_plot)
#
axes = init_fig()

#
for session_id in np.arange(1,9,1):

    #
    psths_avg, psths_shuffled_avg, idx_cells, n_bursts  = get_reward_triggered_psth(session_id, 
                                                                                    c,
                                                                                    window,
                                                                                    idx_cells)
    #
    plot_psth_rois_only(c,
                        psths_avg,
                        psths_shuffled_avg,
                        session_id,
                        axes,
                        start_cell,
                        idx_cells,
                        show_random,
                        smoothing,
                        window
                        )


    # reset axes
    if plot_separately:
        #
        plt.suptitle(c.animal_id+" session: "+str(session_id),fontsize=20)
        #
        plt.savefig('/home/cat/'+str(start_cell)+"_session_"+str(session_id)+'.png',dpi=300)
        plt.close()

        axes = init_fig()

#
plt.suptitle(c.animal_id+" session: "+str(session_id),fontsize=20)
#
plt.savefig('/home/cat/'+str(start_cell)+"_session_"+str(session_id)+'.png',dpi=300)
plt.close()


c.cells_to_plot:  [270 321 223 196]
session:  1  of 8 ,  reward times:  (33,)
session:  2  of 8 ,  reward times:  (41,)
session:  3  of 8 ,  reward times:  (33,)
session:  4  of 8 ,  reward times:  (45,)
session:  5  of 8 ,  reward times:  (64,)
session:  6  of 8 ,  reward times:  (39,)
session:  7  of 8 ,  reward times:  (56,)
session:  8  of 8 ,  reward times:  (56,)


In [126]:
##############################################################
######## COMPUTE REWARD TRIGGERED RESPONSE OF CELLS ##########
##############################################################
window = 30*6
psth_array = []
idx_cells = None
global_order = False
session_ids = np.arange(1,len(c.session_ids),1)

#
n_bursts_array = []
for session_id in session_ids:
#for session_id in [1,7]:

    #
    psths_avg, psths_shuffled_avg, idx_cells, n_bursts  = get_reward_triggered_psth(session_id, 
                                                                                    c,
                                                                                    window,
                                                                                    idx_cells)
    
    #
    n_bursts_array.append(n_bursts)
    
    #
    psth_array.append(psths_avg[idx_cells[::-1]])

    if global_order==False:
        idx_cells = None

#
psth_array = np.array(psth_array)
print (psth_array.shape)

#
vmax = 0.3
sua = plot_multi_session_psth_imshow(c,
                                session_ids,
                                psth_array,
                                window,
                                n_bursts_array,
                                vmax=vmax)


session:  1  of 8 ,  reward times:  (33,)
session:  2  of 8 ,  reward times:  (41,)
session:  3  of 8 ,  reward times:  (33,)
session:  4  of 8 ,  reward times:  (45,)
session:  5  of 8 ,  reward times:  (64,)
session:  6  of 8 ,  reward times:  (39,)
session:  7  of 8 ,  reward times:  (56,)
session:  8  of 8 ,  reward times:  (56,)
(8, 540, 360)


In [5]:
#######################################
#######################################
#######################################

# binarize the ensembel state    
c.plotting = False
c.scale_threshold = 0.05 #                 # we want to decrease the burst detection threshold manually - because the 
                                            # automatic threshold detection is too conservative 
c.dff_threshold = 0.1
c.percentile_threshold = 0.999

#
#print ("TODO: need to add the standaline binairzation code to the binarize2pcalcium package....")
#print (" This is generally a good idea anyways for doing single time sieres extraction")

# right now only binarizing upphase - if want to do negative phase have to invert the signal
c.sample_rate = 30
c.binarize_ensemble_state_multi()


fitting mode to physics:   0%|          | 0/1 [00:00<?, ?it/s]

/home/cat/.conda/envs/bmi/lib/python3.8/site-packages/binarize2pcalcium/binarize2pcalcium.py:4026: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_mode = scipy.stats.mode(F_filtered2)[0]
fitting mode to physics: 100%|██████████| 1/1 [00:00<00:00, 81.99it/s]


cumsum:  [4.63649033e-15 9.27561305e-15 1.39173696e-14 ... 1.00000000e+00
 1.00000000e+00 1.00000000e+00]


binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 1/1 [00:00<00:00, 79.64it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 1/1 [00:00<00:00, 74.52it/s]
fitting mode to physics: 100%|██████████| 1/1 [00:00<00:00, 76.03it/s]


cumsum:  [5.01316877e-10 1.00276836e-09 1.50435447e-09 ... 1.00000000e+00
 1.00000000e+00 1.00000000e+00]


...

binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 1/1 [00:00<00:00, 61.93it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 1/1 [00:00<00:00, 53.58it/s]
fitting mode to physics: 100%|██████████| 1/1 [00:00<00:00, 76.58it/s]


cumsum:  [1.26496650e-17 2.53084175e-17 3.79762638e-17 ... 1.00000000e+00
 1.00000000e+00 1.00000000e+00]


binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 1/1 [00:00<00:00, 63.18it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 1/1 [00:00<00:00, 56.17it/s]
fitting mode to physics: 100%|██████████| 1/1 [00:00<00:00, 66.88it/s]


cumsum:  [4.13738239e-19 8.27809700e-19 1.24221465e-18 ... 1.00000000e+00
 1.00000000e+00 1.00000000e+00]


binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 1/1 [00:00<00:00, 53.34it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 1/1 [00:00<00:00, 40.86it/s]


In [6]:
#######################################################
#######################################################
#######################################################

plt.figure()
t = np.arange(c.sessions[1].e_state.shape[0])/30.

# make viridis colormap for line plotting up to 7 sessions
cmap = matplotlib.cm.get_cmap('viridis')
clrs = cmap(np.linspace(0, 1, 7))

#
for session_id in range(1,8,1):
    temp = c.sessions[session_id].e_state_bin 
    temp2 = c.sessions[session_id].e_state

    plt.plot(t, temp + 2*session_id, 
             label = "bmi "+str(session_id),
             c='black',
             alpha=.5)
    
    plt.plot(t, temp2 + 2*session_id, 
             c=clrs[session_id-1])

#
plt.xlabel("Time (sec)",fontsize=16)

#
plt.xlim(0, t[-1])
plt.yticks([])

#c.sessions[1]
plt.legend()

#
plt.suptitle("Ensemble state + Binarization", fontsize=16)

plt.show()


In [20]:
#######################################
#######################################
#######################################

#
#c.sessions[session_id].thresh = thresh
c.overwrite_spreadsheet = True
c.plotting = True
c.update_spreadsheet = True
c.burst_sign = 1         # -1 or +1
c.process_snippets()           # processes snippets and adds them to the spreassheet 

print ("TO ADD: grab the tones also during the bursts")



# of snippets:  292  # of detected snippets of sign:  1   292
# of bursts:  292  # of rewarded bursts:  46
self.update_spreadsheet:  True
TO ADD: grab the tones also during the bursts


In [22]:
###############################################
###############################################
###############################################

#
#c.time_warp = False
c.plotting = True

#c.use_rewarded_bursts_only = False
c.get_mean_warped_trace()  # this computes the warp trace regardless

snippets_warped:  (46, 30)


In [26]:
################################################
######### VISUALIZE WARPING FUNCITONS ##########
################################################

#
c.use_stretched = True
c.plotting = True
c.burst_to_threshold_only = True                 # this uses the bursts only to the threshold
c.compute_warping_functions()


In [27]:
# this plots the best cell responses to bursts
c.start_idx = 0
c.pre_burst_n_time_pts = 200
c.ca_type = 'DFF'

# 
c.plot_warped_cell_ca_trial_and_burst()

#

  0%|          | 0/354 [00:00<?, ?it/s]

100%|██████████| 354/354 [00:17<00:00, 19.67it/s]


burst_raw:  (354, 46, 400)
post_burst_raw:  (354, 46, 100)
burst_raw_mean:  (354, 400)
(46, 200) (46, 30) (46, 100)
traces_all:  (354, 46, 330)


In [22]:
################################################
######### VISUALIZE WARPING FUNCITONS ##########
################################################
# this plots the best cell responses to bursts
c.start_idx = 0
c.plot_warped_cell_ca_burst_only()

#
print ("SOME TODOS")
print (" 1. Warp only to the threshold - not the peak of burst")
print (" 2. compute the rate map for each cell for burst initiation to threshold/end of peak (2 options)")
print (" 3. compute the rate map for each cell from trial beginning to threshold/end of peak (2 options)")


n_cells:  263 # rewarsds:  39
traces-raw:  (263, 39, 400)
traces mean:  (263, 400)


In [76]:
# plt.figure()

# #
# temp = c.bursts.copy().astype(float)
# temp[temp ==1000] = np.nan
# temp[temp >0] = 1.5
# plt.plot(temp,label='burst')
# # same but with rewarded bursts
# temp = c.bursts_rewarded.copy().astype(float)
# temp[temp !=1] = np.nan

# plt.plot(temp,label='burst rewarded')

# # plot water reward
# plt.plot(c.water_reward*1.3,label='water')

# #
# plt.plot(c.e_state,label='ensemble state')

# plt.legend()
# plt.show()
